In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.datasets import fetch_california_housing
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("C:/Users/AKASH HN/Downloads/Dataset/Hitters.csv")
data.head(5)

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [2]:
data.isnull().sum()

Unnamed: 0     0
AtBat          0
Hits           0
HmRun          0
Runs           0
RBI            0
Walks          0
Years          0
CAtBat         0
CHits          0
CHmRun         0
CRuns          0
CRBI           0
CWalks         0
League         0
Division       0
PutOuts        0
Assists        0
Errors         0
Salary        59
NewLeague      0
dtype: int64

In [3]:
data['Salary'].fillna(data['Salary'].mean(),inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  322 non-null    object 
 1   AtBat       322 non-null    int64  
 2   Hits        322 non-null    int64  
 3   HmRun       322 non-null    int64  
 4   Runs        322 non-null    int64  
 5   RBI         322 non-null    int64  
 6   Walks       322 non-null    int64  
 7   Years       322 non-null    int64  
 8   CAtBat      322 non-null    int64  
 9   CHits       322 non-null    int64  
 10  CHmRun      322 non-null    int64  
 11  CRuns       322 non-null    int64  
 12  CRBI        322 non-null    int64  
 13  CWalks      322 non-null    int64  
 14  League      322 non-null    object 
 15  Division    322 non-null    object 
 16  PutOuts     322 non-null    int64  
 17  Assists     322 non-null    int64  
 18  Errors      322 non-null    int64  
 19  Salary      322 non-null    f

In [4]:
dms = pd.get_dummies(data[['League','Division','NewLeague']])
dms

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
0,True,False,True,False,True,False
1,False,True,False,True,False,True
2,True,False,False,True,True,False
3,False,True,True,False,False,True
4,False,True,True,False,False,True
...,...,...,...,...,...,...
317,False,True,True,False,False,True
318,True,False,True,False,True,False
319,True,False,False,True,True,False
320,True,False,True,False,True,False


In [5]:
x = data.drop(['Salary','League','Division','NewLeague','Unnamed: 0'],axis = 1)
y = data['Salary']
X = pd.concat([x,dms[['League_N','Division_W','NewLeague_N']]],axis = 1)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [7]:
lasso = Lasso().fit(X_train,y_train)
y_pred = lasso.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

331.0530864986833

In [8]:
r2_score(y_test, y_pred)

0.4849194408652657

In [9]:
lasso.intercept_

356.55942004312595

In [10]:
lasso.coef_

array([-1.66697858e+00,  4.66316611e+00,  4.85799425e+00,  3.44953963e-02,
       -1.25410138e+00,  4.59950472e+00, -6.17260605e+00, -1.86885232e-01,
        4.04782323e-01, -4.46628045e-01,  9.57693202e-01,  8.15437140e-01,
       -7.06230761e-01,  2.19972348e-01,  2.63588648e-01, -3.35750884e+00,
        2.45623757e+01, -1.05414474e+02, -0.00000000e+00])

In [11]:
lasso_cv = LassoCV(alphas=np.random.randint(1,1000,100),cv = 10,max_iter = 10000,n_jobs = -1).fit(X_train,y_train)
lasso_cv.alpha_

228

In [12]:
lasso_tuned = Lasso(alpha= 239).fit(X_train,y_train)
y_pred = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

345.1535660872464

In [13]:
pd.Series(lasso_tuned.coef_,index = X_train.columns)

AtBat         -0.582139
Hits           1.831748
HmRun          0.000000
Runs           0.000000
RBI            0.000000
Walks          2.584050
Years         -0.000000
CAtBat        -0.304201
CHits          0.838761
CHmRun         0.000000
CRuns          0.653548
CRBI           0.598505
CWalks        -0.304891
PutOuts        0.194354
Assists        0.089743
Errors        -0.000000
League_N       0.000000
Division_W    -0.000000
NewLeague_N    0.000000
dtype: float64